In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.datasets import make_moons

X,y=make_moons(n_samples=10000,noise=0.4,random_state=42)

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

log_clf=LogisticRegression(solver='lbfgs', random_state=42)
rnd_clf=RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf=SVC(gamma='scale',probability=True, random_state=42)
 
voting_clf=VotingClassifier(
estimators=[('lr',log_clf),('rf',rnd_clf),('svc',svm_clf)],
voting='soft')
voting_clf.fit(X_train,y_train)

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(probability=True, random_state=42))],
                 voting='soft')

In [11]:
from sklearn.metrics import accuracy_score

for clf in (log_clf,rnd_clf,svm_clf,voting_clf):
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    print(clf.__class__.__name__,accuracy_score(y_test,y_pred))

LogisticRegression 0.8415
RandomForestClassifier 0.8505
SVC 0.874
VotingClassifier 0.8705


In [12]:
#배깅과 페이스팅

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf=BaggingClassifier(
    DecisionTreeClassifier(),n_estimators=500,
    max_samples=100,bootstrap=True,n_jobs=-1)
bag_clf.fit(X_train,y_train)
y_pred=bag_clf.predict(X_test)

In [13]:
print(accuracy_score(y_test,y_pred))

0.871


In [14]:
tree_clf=DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train,y_train)
y_pred_tree=tree_clf.predict(X_test)
print(accuracy_score(y_test,y_pred_tree))

0.8145


In [16]:
bag_clf=BaggingClassifier(
DecisionTreeClassifier(),n_estimators=500,
bootstrap=True,n_jobs=-1,oob_score=True)

bag_clf.fit(X_train,y_train)
bag_clf.oob_score_

0.836125

In [17]:
from sklearn.metrics import accuracy_score

y_pred=bag_clf.predict(X_test)
accuracy_score(y_test,y_pred)

0.8525

In [18]:
#랜덤 포레스트

from sklearn.ensemble import RandomForestClassifier

rnd_clf=RandomForestClassifier(n_estimators=500,max_leaf_nodes=16,n_jobs=-1)
rnd_clf.fit(X_train,y_train)

y_pred_rf=rnd_clf.predict(X_test)

In [19]:
accuracy_score(y_test,y_pred_rf)

0.8715

In [20]:
#특성 중요도

from sklearn.datasets import load_iris

iris=load_iris()
rnd_clf=RandomForestClassifier(n_estimators=500,n_jobs=-1)
rnd_clf.fit(iris['data'],iris['target'])
for name,score in zip(iris['feature_names'],rnd_clf.feature_importances_):
    print(name,score)


sepal length (cm) 0.09629339747529801
sepal width (cm) 0.023103494466906457
petal length (cm) 0.41287818863316555
petal width (cm) 0.46772491942462996


In [21]:
#아다부스팅

from sklearn.ensemble import AdaBoostClassifier

ada_clf=AdaBoostClassifier(
DecisionTreeClassifier(max_depth=1),n_estimators=200,
algorithm='SAMME.R',learning_rate=0.5)

ada_clf.fit(X_train,y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

In [22]:
y_pred_ada=ada_clf.predict(X_test)
accuracy_score(y_test,y_pred_ada)

0.861

In [27]:
#그레이디언트 부스팅
import numpy as np

np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)

from sklearn.tree import DecisionTreeRegressor

tree_reg1=DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)

y2=y-tree_reg1.predict(X)
tree_reg2=DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X,y2)

y3=y2-tree_reg2.predict(X)
tree_reg3=DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X,y3)

DecisionTreeRegressor(max_depth=2)

In [28]:
X_new = np.array([[0.8]])

In [29]:
y_pred=sum(tree.predict(X_new) for tree in (tree_reg1,tree_reg2,tree_reg2))

In [30]:
y_pred

array([0.60839441])

In [3]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt=GradientBoostingRegressor(max_depth=2,n_estimators=3,learning_rate=1.0)
gbrt.fit(X,y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train,X_val,y_train,y_val=train_test_split(X,y)

gbrt=GradientBoostingRegressor(max_depth=2,n_estimators=120)
gbrt.fit(X_train,y_train)

errors=[mean_squared_error(y_val,y_pred)
       for y_pred in gbrt.staged_predict(X_val)]

bst_n_estimators=np.argmin(errors)+1

gbrt_best=GradientBoostingRegressor(max_depth=2,n_estimators=bst_n_estimators)
gbrt_best.fit(X_train,y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=85)

In [6]:
gbrt=GradientBoostingRegressor(max_depth=2,warm_start=True)

min_val_error=float('inf')
error_going_up=0
for n_estimators in range(1,120):
    gbrt.n_estimators=n_estimators
    gbrt.fit(X_train,y_train)
    y_pred=gbrt.predict(X_val)
    val_error=mean_squared_error(y_val,y_pred)
    if val_error<min_val_error:
        min_val_error=val_error
        error_going_up=0
    else:
        error_going_up+=1
        if error_going_up==5:
            break

In [9]:
import xgboost

xgb_reg=xgboost.XGBRegressor()
xgb_reg.fit(X_train,y_train)
y_pred=xgb_reg.predict(X_val)

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [11]:
xgb_reg.fit(X_train,y_train,eval_set=[(X_val,y_val)],early_stopping_rounds=2)
y_pred=xgb_reg.predict(X_val)

[0]	validation_0-rmse:0.41542
[1]	validation_0-rmse:0.36719
[2]	validation_0-rmse:0.34022
[3]	validation_0-rmse:0.32705
[4]	validation_0-rmse:0.32081
[5]	validation_0-rmse:0.31705
[6]	validation_0-rmse:0.31590
[7]	validation_0-rmse:0.31581
[8]	validation_0-rmse:0.31552
[9]	validation_0-rmse:0.31521
[10]	validation_0-rmse:0.31522
[11]	validation_0-rmse:0.31499
[12]	validation_0-rmse:0.31540
[13]	validation_0-rmse:0.31540


In [ ]:
#스태킹